## Setup - Import modules and setup database connections

In [ ]:
# import necessary modules and setup database connection

import json
import logging
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


# Load settings from settings.json
settings_file = '../settings.json'
with open(settings_file) as f:
    settings = json.load(f)

# Database setup
DATABASE_URL = settings['sqlalchemy_database_uri']
engine = create_engine(DATABASE_URL, echo=False)
Session = sessionmaker(bind=engine)

# move current directory to parent directory
import os
os.chdir('..')

from sandbox.models import Sample, Tag, Analysis, sample_tag
from gi.models import Prototypes, Ingredient, Candidate
session = Session()

## Count all major entities

In [ ]:
# counts of all the major entities

count_samples = session.query(Sample).count()
count_Tags = session.query(Tag).count()
count_Analyses = session.query(Analysis).count()
count_Prototypes = session.query(Prototypes).count()
count_Ingredients = session.query(Ingredient).count()
count_Candidates = session.query(Candidate).count()
print(f"Number of samples in the database: {count_samples}")
print(f"Number of tags in the database: {count_Tags}")
print(f"Number of analyses in the database: {count_Analyses}")
print(f"Number of prototypes in the database: {count_Prototypes}")
print(f"Number of ingredients in the database: {count_Ingredients}")
print(f"Number of candidates in the database: {count_Candidates}")



## Get tags and counts

In [ ]:
# get tags and their associated sample counts
tags = session.query(Tag).all()
tag_sample_counts = {}
for tag in tags:
    sample_count = session.query(Sample).join(sample_tag).filter(sample_tag.c.tag_id == tag.id).count()
    tag_sample_counts[(tag.key, tag.value)] = sample_count
print("\nTag Sample Counts:")
for tag_value, sample_count in tag_sample_counts.items():
    print(f"Tag: {tag_value}, Sample Count: {sample_count}")

In [ ]:
# get all samples asscoiated with a specific tag
tag_key = 'class'
tag_value = 'wmi'
samples_with_tag = session.query(Sample).join(sample_tag).join(Tag).filter(Tag.key == tag_key, Tag.value == tag_value).all()
print(f"\nSamples with tag ({tag_key}={tag_value}):")
for sample in samples_with_tag:
    print(f"Sample ID: {sample.sha256}, Name: {sample.filepath}")

In [ ]:
# get all analyses and with a specific tag
tag_key = 'class'
tag_value = 'wmi'
analyses_with_tag = session.query(Analysis).join(Sample).join(sample_tag).join(Tag).filter(Tag.key == tag_key, Tag.value == tag_value).all()
print(f"\nAnalyses with tag ({tag_key}={tag_value}):")
for analysis in analyses_with_tag:
    print(f"Analysis ID: {analysis.id}, Sample ID: {analysis.sample}, Analysis Status: {analysis.status}") 

In [ ]:
# get counts of all statuses of analyses
from sqlalchemy import func
analysis_status_counts = session.query(Analysis.status, func.count(Analysis.id)).group_by(Analysis.status).all()
print("\nAnalysis Status Counts:")
for status, count in analysis_status_counts:
    print(f"Status: {status}, Count: {count}")

In [ ]:
# get analysis by sample sha256
sample_sha256 = 'f2a839f3eac858ddb450a162a9faa6fe54391fb0f0b0c715584cffe36db6e191'
analysis_for_sample = session.query(Analysis).join(Sample).filter(Sample.sha256 == sample_sha256).all()
print(f"\nAnalyses for sample with SHA256 {sample_sha256}:")
for analysis in analysis_for_sample:
    print(f"Analysis ID: {analysis.id}, Status: {analysis.status}")
if not analysis_for_sample:
    print("No analyses found for this sample.")

In [ ]:
# get analysis by its ID
analysis_id = 17849  # replace with desired analysis ID
analysis = session.query(Analysis).filter(Analysis.id == analysis_id).first()
if analysis:
    print(f"\nAnalysis ID: {analysis.id}, Status: {analysis.status}, Sample ID: {analysis.sample}")
else:
    print(f"\nNo analysis found with ID {analysis_id}.")

In [ ]:
# get candidates
candidates = session.query(Candidate).all()
print("\nCandidates:")
for candidate in candidates:
    print(f"Candidate ID: {candidate.hash}, Status: {candidate.status}, F1: {candidate.F1}, F2: {candidate.F2}, F3: {candidate.F3}, Analysis ID: {candidate.analysis_id}")

In [ ]:
# update status of a candidate
candidate_hash = '76b6cc1637d9e65dae68fd71f00883652bd7914f47016f7189fb451ec08e187e'  # replace with actual candidate hash
candidate = session.query(Candidate).filter(Candidate.hash == candidate_hash).first()
if candidate:
    candidate.status = 0  # replace with desired status
    session.commit()
    print(f"\nUpdated candidate {candidate_hash} to status '0'.")
else:
    print(f"\nNo candidate found with hash {candidate_hash}.")